In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

from darts import TimeSeries
from darts.metrics import mape
from darts.models import ExponentialSmoothing
from darts.dataprocessing.transformers import Scaler
from darts.utils.statistics import plot_residuals_analysis

import yfinance as yf

In [ ]:
symbol = '^GSPC' # input
start = '2012-01-01' # input 
end = '2022-07-21'  # input
df = yf.download(symbol, start, end)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(df['Close'])
plt.ylabel('Price')
plt.show()

In [ ]:
dfm = df.resample('W').mean()
dfm.head()

In [ ]:
dfm = dfm.reset_index()
series = TimeSeries.from_dataframe(dfm, 'Date', 'Close')

In [ ]:
#split into train and val
train, val = series[:-100], series[-100:]
print(len(dfm))
model = ExponentialSmoothing()
model.fit(train)
prediction = model.predict(len(val), num_samples=1000)

In [ ]:
series.plot()
prediction.plot(label='forecast', low_quantile=0.05, high_quantile=0.95)
plt.legend()

In [ ]:
from darts.models import ExponentialSmoothing, TBATS, AutoARIMA, Theta

def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print("model {} obtains MAPE: {:.2f}%".format(model, mape(val, forecast)))

eval_model(ExponentialSmoothing())
eval_model(TBATS())
eval_model(AutoARIMA())
eval_model(Theta())

In [ ]:
model_es = ExponentialSmoothing()
historical_fcast_es = model_es.historical_forecasts(
    series, start=0.6, forecast_horizon=3, verbose=True
)

series.plot(label="data")
historical_fcast_es.plot(label="backtest 3-weeks ahead forecast (Exp. Smoothing)")
print("MAPE = {:.2f}%".format(mape(historical_fcast_es, series)))

In [ ]:
import darts
from darts import TimeSeries
from darts.metrics import mape
from darts.models import ExponentialSmoothing,TCNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.statistics import plot_residuals_analysis

In [ ]:
prob_model = TCNModel(
    input_chunk_length=14*2,
    output_chunk_length=1,
    num_layers=2,
    dropout=0,
    likelihood=darts.utils.likelihood_models.GaussianLikelihood(),
    n_epochs=1500,
)

In [ ]:
prob_model.fit(train.astype(np.float32))

In [ ]:
pred = prob_model.predict(10, num_samples=100)

In [ ]:
pred.plot(low_quantile=0.05, high_quantile=0.95, color='blue')

In [ ]:
pred = prob_model.predict(75, num_samples=100)

In [ ]:
pred.plot(low_quantile=0.05, high_quantile=0.95, color='blue')